In [1]:
import warnings
warnings.filterwarnings('ignore')

## Hyper Parameter Tuning

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

In [3]:
iris = load_iris()
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

__TrainTest Split__

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target) # (flattened 2d array,1d array)

__SVM Model__

In [5]:
from sklearn.svm import SVC
SVMmodel = SVC(kernel = 'rbf',C=30,gamma='auto')
SVMmodel.fit(X_train,y_train)
SVMmodel.get_params()

{'C': 30,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'auto',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [6]:
SVMmodel.score(X_test,y_test)

0.9473684210526315

The problem with this approach is that when we reload the train test split the score changes.
To tackle with this issue we use cross validation from sklearn

__Cross Validation__

In [7]:
from sklearn.model_selection import cross_val_score  # default n_splits= 5.Train test split is not required for C-validation

In [8]:
cross_val_score(SVC(kernel = 'linear',C=10,gamma='auto'),iris.data,iris.target) # Takes (estimator,X,y)
# (splits into train and test data internally, and gives the score.)
# Details explained in Cross validation Folder

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [9]:
cross_val_score(SVC(kernel = 'rbf',C=10,gamma='auto'),iris.data,iris.target) # Takes (estimator,X,y)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [10]:
cross_val_score(SVC(kernel = 'rbf',C=20,gamma='auto'),iris.data,iris.target) # Takes (estimator,X,y)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

__Cross Validation using: for loop__

In [11]:
#Trial1: Adding values to python dictionary
dictionary= {}
dictionary['Sid']= 98
dictionary

{'Sid': 98}

In [12]:
#Trial2: Adding values to python dictionary
kernels = ['rbf','linear']
dictionary= {}
dictionary[kernels[0]]= 98
dictionary[kernels[1]]= 100
dictionary

{'rbf': 98, 'linear': 100}

In [13]:
kernels = ['rbf','linear'] # list of kernels
C=[1,10,20]                # list of C values
avg_scores = {}            # dictionary 
for i in kernels:
    for j in C:
        cv_scores = cross_val_score(SVC(kernel = i,C=j,gamma='auto'),iris.data,iris.target) 
        avg_scores[i + '_' + str(j)] = np.average(cv_scores) #averages the 5 scores and adds to dictionary
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

__Sklearns Grid Search CV__

In [14]:
from sklearn.model_selection import GridSearchCV #takes (estimator, param_grid, cv)
clf = GridSearchCV(SVC(gamma='auto'),
                   {'C':[1,10,20], 'kernel' : ['rbf','linear']},
                   cv = 5)

clf.fit(iris.data,iris.target)
clf.cv_results_      #results generated are key value pairs.Not easy to view. Creating pandas dataframe of results below.

{'mean_fit_time': array([0.00120149, 0.00100107, 0.0010006 , 0.00100064, 0.00100069,
        0.00100064]),
 'std_fit_time': array([3.99306654e-04, 1.22872340e-06, 9.53674316e-08, 0.00000000e+00,
        9.53674316e-08, 0.00000000e+00]),
 'mean_score_time': array([0.00059996, 0.00020013, 0.00100079, 0.00020008, 0.        ,
        0.00020008]),
 'std_score_time': array([4.89862743e-04, 4.00257111e-04, 1.16800773e-07, 4.00161743e-04,
        0.00000000e+00, 4.00161743e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'ker

In [15]:
grid_df = pd.DataFrame(clf.cv_results_ )  #converting the above generated results to dataframe
grid_df   # returned whole lot of different parameter  values 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001201,3.993067e-04,0.000600,4.898627e-04,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001001,1.228723e-06,0.000200,4.002571e-04,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001001,9.536743e-08,0.001001,1.168008e-07,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001001,0.000000e+00,0.000200,4.001617e-04,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001001,9.536743e-08,0.000000,0.000000e+00,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001001,0.000000e+00,0.000200,4.001617e-04,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [16]:
grid_df = grid_df[['param_C','param_kernel','mean_test_score']] # selecting the required columns only
grid_df

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [17]:
dir(clf)  # some other attributes we can try

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_features_

In [18]:
clf.best_score_ #returns the best score out of all the combinations

0.9800000000000001

In [19]:
clf.best_params_ # returns the best combination

{'C': 1, 'kernel': 'rbf'}

__RandomizedSearchCV__

In [20]:
from sklearn.model_selection import RandomizedSearchCV
clfrandom = RandomizedSearchCV(SVC(gamma='auto'),
                   {'C':[1,10,20], 'kernel' : ['rbf','linear']},
                   cv = 5,
                    n_iter = 3)  # most important parameter of RandomizedSearchCV (decides the number of random outputs)


clfrandom.fit(iris.data,iris.target)
clfrandom.cv_results_      #results generated are key value pairs.Not easy to view. Creating pandas dataframe of results below.

{'mean_fit_time': array([0.00100155, 0.00080037, 0.00100107]),
 'std_fit_time': array([7.13664510e-07, 4.00185937e-04, 6.28991411e-07]),
 'mean_score_time': array([0.00059981, 0.00040026, 0.        ]),
 'std_score_time': array([0.00048975, 0.00049021, 0.        ]),
 'param_kernel': masked_array(data=['linear', 'linear', 'rbf'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[10, 1, 20],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'linear', 'C': 10},
  {'kernel': 'linear', 'C': 1},
  {'kernel': 'rbf', 'C': 20}],
 'split0_test_score': array([1.        , 0.96666667, 0.96666667]),
 'split1_test_score': array([1., 1., 1.]),
 'split2_test_score': array([0.9       , 0.96666667, 0.9       ]),
 'split3_test_score': array([0.96666667, 0.96666667, 0.96666667]),
 'split4_test_score': array([1., 1., 1.]),
 'mean_test_score': array([0.97333333, 0.98   

In [21]:
rand_df = pd.DataFrame(clfrandom.cv_results_ )  #converting the above generated results to dataframe
rand_df   # returned only 2 randomly generated reults. 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001002,7.136645e-07,0.0006,0.00049,linear,10,"{'kernel': 'linear', 'C': 10}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,2
1,0.000800,4.001859e-04,0.0004,0.00049,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001001,6.289914e-07,0.0000,0.00000,rbf,20,"{'kernel': 'rbf', 'C': 20}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,3


In [22]:
rand_df = rand_df[['param_C','param_kernel','mean_test_score']] # selecting the required columns only
rand_df

,param_C,param_kernel,mean_test_score
0,10,linear,0.973333
1,1,linear,0.980000
2,20,rbf,0.966667


In [23]:
clfrandom.best_score_ 

0.9800000000000001

In [24]:
clfrandom.best_params_ 

{'kernel': 'linear', 'C': 1}

## Best Model Selection - with Hyper parameter Tuning

In [25]:
list(range(1,10)) #This can also be used further in model_params to automate the parameters in range.
# for C
# for n_estimators

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [26]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [27]:
model_params = {                           # creating model poarameters
    'svm': {
        'model': SVC(gamma='auto'),
        'params': {
            'C':[1,10,20],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest':{
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [1,5,10]
        }
    }   
}

In [28]:
model_params

{'svm': {'model': SVC(gamma='auto'),
  'params': {'C': [1, 10, 20], 'kernel': ['rbf', 'linear']}},
 'random_forest': {'model': RandomForestClassifier(),
  'params': {'n_estimators': [1, 5, 10]}},
 'logistic_regression': {'model': LogisticRegression(),
  'params': {'C': [1, 5, 10]}}}

In [29]:
df_params = pd.DataFrame(model_params) # creating and viewing the newly created parameters in dataframe.
df_params

,svm,random_forest,logistic_regression
model,SVC(gamma='auto'),RandomForestClassifier(),LogisticRegression()
params,"{'C': [1, 10, 20], 'kernel': ['rbf', 'linear']}","{'n_estimators': [1, 5, 10]}","{'C': [1, 5, 10]}"


In [30]:
#trial1 :
for i,j in model_params.items(): 
    print(i)     # returns the estimators
#     print(j)

svm
random_forest
logistic_regression


In [31]:
#trial2
for i,j in model_params.items():
#     print(i)
    print(j)    # returns the parameter grid

{'model': SVC(gamma='auto'), 'params': {'C': [1, 10, 20], 'kernel': ['rbf', 'linear']}}
{'model': RandomForestClassifier(), 'params': {'n_estimators': [1, 5, 10]}}
{'model': LogisticRegression(), 'params': {'C': [1, 5, 10]}}


In [32]:
#trial3 - estimators
for i,j in model_params.items():
#     print(i)
    print(j['model'])

SVC(gamma='auto')
RandomForestClassifier()
LogisticRegression()


In [33]:
#trial4 - parameters
for i,j in model_params.items():
#     print(i)
    print(j['params'])

{'C': [1, 10, 20], 'kernel': ['rbf', 'linear']}
{'n_estimators': [1, 5, 10]}
{'C': [1, 5, 10]}


__Finding the best model with their best scores and best parameters__

In [38]:
from sklearn.model_selection import GridSearchCV #takes (estimator, param_grid, cv)

In [34]:
scores = []
for i,j in model_params.items():
    clf = GridSearchCV(j['model'], j['params'],cv =5)
    clf.fit(iris.data,iris.target)
    scores.append({
        'MODEL': i,
        'BEST SCORE': clf.best_score_,
        'BEST PARAMS': clf.best_params_
    })
scores

[{'MODEL': 'svm',
  'BEST SCORE': 0.9800000000000001,
  'BEST PARAMS': {'C': 1, 'kernel': 'rbf'}},
 {'MODEL': 'random_forest',
  'BEST SCORE': 0.9666666666666668,
  'BEST PARAMS': {'n_estimators': 5}},
 {'MODEL': 'logistic_regression',
  'BEST SCORE': 0.9800000000000001,
  'BEST PARAMS': {'C': 5}}]

In [37]:
type(scores) #this is a list, with dictionary inside it.

list

In [35]:
best_model_df = pd.DataFrame(scores) # creating dataframe from the above scores 
best_model_df

,MODEL,BEST SCORE,BEST PARAMS
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 5}
2,logistic_regression,0.980000,{'C': 5}
